In [2]:
import psycopg2
import pandas as pd
import sys
sys.path.append("..")
import credentials
from sqlalchemy import create_engine
engine = create_engine(credentials.postgres)
import numpy as np
from datetime import datetime
# This makes plots render inline

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [4]:
# Basic Connection and Query
connection = engine.connect()

In [5]:
key_codes = pd.read_csv('key_codes.csv').set_index('code').key

# Calculate Dwell Times

In [6]:
def get_dwell_sequence(connection=connection):
    people = {}
    threshold = 600
    # Calculate Dwells
    result = connection.execute("SELECT * FROM strokes ORDER BY key_time")
    dwells = []
    for row in result:
        # ('1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f', 1509862331813, '3675', '40964', 'd')
        row = (row[0], row[1], int(row[2]), int(row[3]), row[4])
        # Assure Users are in Dictionaries
        if row[0] not in people:
            people[row[0]] = {}
            
        if row[2] not in people[row[0]] and row[4] == 'd':
            people[row[0]][row[2]] = row[1]
        elif row[2] in people[row[0]] and row[4] == 'd':
            people[row[0]][row[2]] = row[1]
        elif row[2] in people[row[0]] and row[4] == 'u':
            dwell = row[1] - people[row[0]][row[2]]
            if dwell < threshold:
                try:
                    dwells.append({
                        'person': row[0],
                        'key_code': row[2],
                        'key': key_codes[row[2]],
                        'down': people[row[0]][row[2]],
                        'up': row[1],
                        'dwell': dwell
                    })
                except Exception as e:
                    pass
                
            del(people[row[0]][row[2]])
    return pd.DataFrame(dwells)


def get_average_dwells(dwells):
    average_dwells = {}
    for p in dwells:
        average_dwells[p] = {}
        for k in dwells[p]:
            average_dwells[p][k] = np.mean(dwells[p][k])
    return average_dwells

In [7]:
dwell_df = get_dwell_sequence()

In [8]:
dwell_df

,down,dwell,key,key_code,person,up
0,1509858869909,60,V,47,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858869969
1,1509858870179,176,E,18,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858870355
2,1509858870315,101,R,19,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858870416
3,1509858870459,85,A,30,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858870544
4,1509858870636,133,G,34,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858870769
5,1509858871213,124,E,18,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858871337
6,1509858871831,64,A,30,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858871895
7,1509858871970,92,V,47,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858872062
8,1509858872076,112,E,18,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858872188
9,1509858872173,63,R,19,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509858872236


In [8]:
dwell_df.to_csv('output/dwell_times.csv', index=False)

# Calculate Flight Time

In [ ]:
def get_flight_sequence(connection=connection):
    people = {}
    threshold = 1500
    result = connection.execute("SELECT * FROM strokes WHERE direction='d' ORDER BY key_time")
    flights = []
    for row in result:
        # ('1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f', 1509862331813, '3675', '40964', 'd')
        row = (row[0], row[1], int(row[2]), int(row[3]), row[4])
        # Assure Users are in Dictionaries
        if row[0] not in people:
            people[row[0]] = None
        # Get Last key of user
        last = people[row[0]]
        # Assure last key and next key is in flights
        if last is not None:
            # Add next key and set last
            flight = row[1] - last[1]
            if flight <= threshold:
                try:
                    flights.append({
                        'person': row[0],
                        'start_key_code': last[2],
                        'start_key': key_codes[last[2]],
                        'start': last[1],
                        'finish_key_code': row[2],
                        'finish_key': key_codes[row[2]],
                        'finish': row[1],
                        'flight': flight
                    })
                except Exception as e:
                    pass
        people[row[0]] = row
    return pd.DataFrame(flights)

def get_average_flights(flights):
    average_flights = {}
    for p in flights:
        average_flights[p] = {}
        for a in flights[p]:
            average_flights[p][a] = {}
            for b in flights[p][a]:
                average_flights[p][a][b] = np.mean(flights[p][a][b])
    return average_flights

In [ ]:
flight_df = get_flight_sequence()

In [ ]:
flight_df.to_csv('output/flight_times.csv', index=False)

In [ ]:
connection.close()

In [9]:
pd.read_sql_query('SELECT * FROM strokes LIMIT 100', engine)

,user_id,key_time,key_code,modifiers,direction
0,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862331813,3675,40964,d
1,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862331929,20,40964,d
2,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862332021,20,40964,u
3,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862332077,3675,40960,u
4,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862332204,25,40960,d
5,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862332308,25,40960,u
6,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862332414,24,40960,d
7,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862332509,31,40960,d
8,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862332512,24,40960,u
9,1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f,1509862332585,31,40960,u
